<h1 align=center style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
تحلیل زمانی
</font>
</h1>


In [19]:
import pandas as pd
import matplotlib.pyplot as plt

housing_extended = pd.read_csv('../3_add_new_features/housing_extended.csv', encoding='gbk')

<p dir=rtl style="direction: rtl;text-align: right;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
می‌خواهیم گزارشی تهیه کنیم و مناطق مختلف شهر را از زوایای گوناگون مورد بررسی قرار دهیم. 
</font>
</p>

<h2 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
قسمت اول
</font>
</h2>
<p dir=rtl style="direction: rtl;text-align: right;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
دیتافریمی به نام <code>districts_df</code> بسازید و <code>index</code> آنرا برابر ستون <code>district</code> قرار دهید. برای هر منطقه موارد زیر را حساب کنید:
<ul dir=rtl>

  <li>
pricePerSquare mean
<br>
متوسط قیمت هر متر مربع
</li>
<br>

  <li>
% houses with elevator
<br>
درصد خانه‌هایی که آسانسور دارند
</li>
<br>

  <li>
% houses with subway
<br>
درصد خانه‌هایی که به مترو دسترسی دارند
</li>
<br>

  <li>
square mean
<br>
متوسط متراژ خانه‌ها
</li>
<br>
  <li>
building age mean
<br>
متوسط عمر خانه‌ها (به سال)
<br>
(عمر هر خانه تا سال ۲۰۲۱ محاسبه می‌شود.)
</li>
<br>

  <li>
frequent buildingStructure
<br>
رایج‌ترین نوع ساختمان
</li>
<br>
</ul>
</font>
</p>

In [15]:
housing_extended["age"] = 2021 - housing_extended["constructionTime"]
dct = {"pricePerSquare mean" : housing_extended.groupby("district")["pricePerSquare"].mean(),
       "% houses with elevator" : housing_extended[housing_extended["elevator"] == "has elevator"].groupby("district").size() / housing_extended.groupby("district").size() * 100,
       "% houses with subway" : housing_extended[housing_extended["subway"] == "has subway"].groupby("district").size() / housing_extended.groupby("district").size() * 100,
       "square mean" : housing_extended.groupby("district")["square"].mean(),
       "building age mean" : housing_extended.groupby("district")["age"].mean(),
       "frequent buldingStructure" : housing_extended.groupby("district")["buildingStructure"].max()
}
housing_extended.drop(columns="age")

districts_df = pd.DataFrame(dct)


In [17]:
districts_df.head(1)

pricePerSquare mean  % houses with elevator  % houses with subway  \
district                                                                      
1                 5981.774965                56.21655             93.488404   

          square mean  building age mean frequent buldingStructure  
district                                                            
1           67.047856          25.151709                   unknown

In [18]:

districts_df.set_index('district', inplace=True)

KeyError: "None of ['district'] are in the columns"

<p dir=rtl style="direction: rtl;text-align: right;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
خط اول خروجی شما باید به شکل زیر باشد. (سطرها را بر اساس شماره منطقه مرتب کنید.)
</font>
</p>


|   district |   pricePerSquare mean |   % houses with elevator |   % houses with subway |   square mean |   buildeng age mean | frequent buildingStructure   |
|-----------:|----------------------:|-------------------------:|-----------------------:|--------------:|--------------------:|:-----------------------------|
|          1 |               5981.77 |                  56.216550 |                93.4884 |       67.0479 |             25.1517 | steel-concrete composite     |

<p dir=rtl style="direction: rtl;text-align: right;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
تحلیل داده‌ها بر اساس ترتیب زمانی آنها در برخی حوزه‌ها کاربرد بسیاری دارد. برای مثال، پیش‌بینی قیمت در بورس، پیش‌بینی قیمت طلا، بررسی حجم فروش اجناس در بازه‌های زمانی مختلف و غیره. زمان انجام معاملات هر خانه در این دیتافریم مشخص شده است و ما می‌توانیم چنین تحلیل‌هایی را بر روی آن انجام دهیم.
</font>
</p>

<p dir=rtl style="direction: rtl;text-align: right;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
اولین کاری که باید انجام دهیم، تبدیل ستون tradeTime به فرمت زمانی می‌باشد. برای این‌کار از تابع <code>to_datetime</code> استفاده می‌کنیم.
</font>
</p>

In [ ]:
housing_extended['tradeTime'] = pd.to_datetime(housing_extended['tradeTime'])

<h2 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
قسمت دوم
</font>
</h2>
<p dir=rtl style="direction: rtl;text-align: right;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
طبق نظر افراد محلی، چند ماه است که قیمت خانه در حال کاهش است و پیش‌بینی می‌شود این روند ادامه یابد. در این تمرین می‌خواهیم صحت این نظریه را بررسی کنیم.
<ul dir=rtl>
  <li>متد <code>resample</code> متدی قدرتمند برای کار با سری‌های زمانی است. با این متد می‌توان سطرها را بر اساس بازه زمانی گروه‌بندی کرد. پارامتر <code>rule</code> بازه مورد نظر و پارامتر <code>on</code> ستونی که زمان در آن ذخیره شده را مشخص می‌کند.</li>
  <li>یک عکس شامل یک صفحه محور بسازید و اندازه طول و عرض آن را برابر ۸ اینچ قرار دهید.</li>
  <li>با استفاده از متد <code>resample</code> میانگین قیمت هر متر مربع در معاملات ماهانه را حساب کنید. قبل از سال ۲۰۱۰ زیر ده معامله انجام شده است. معاملات بعد از سال ۲۰۱۰ را وارد محاسبات خود کنید.
</li>
  <li>
با استفاده از رابط پانداس، اطلاعات به دست آمده را روی صفحه محوری که پیش از این ساختید رسم کنید. رنگ خط را برابر <code>crimson</code> قرار دهید.
</li>

<li>
تنظیمات را طوری تغییر دهید که نمودار خروجی شما دقیقا مانند شکل زیر شود.
</li>

</ul>
</font>
</p>


<p style="text-align:center;"><img src="fig1.png"></p>



In [ ]:
fig1 , ax1 = None

<h2 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
قسمت سوم
</font>
</h2>
<p dir=rtl style="direction: rtl;text-align: right;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
شایعه دیگری که می‌خواهیم بررسی کنیم، کاهش <b> تعداد </b> معاملات خانه‌های نزدیک مرکز شهر است. این نمودار مانند نمودار قبلی است و این تغییرات را باید در آن ایجاد کنید.
<ul dir=rtl>

  <li>
بازه زمانی به جای ماه، ۳ ماه می‌باشد و علاوه بر اینکه فقط معاملات بعد از ۲۰۱۰ باید بررسی شوند، معاملات خانه‌هایی که فاصله آن‌ها از مرکز بیشتر یا مساوی ۱۵ کیلومتر است هم باید حذف شوند.
</li>
  <li>
رنگ خط را برابر <code>orange</code> قرار دهید.
</li>

<li>
تنظیمات را طوری تغییر دهید که نمودار خروجی شما دقیقا مانند شکل زیر شود.
</li>

<li>
نکته: ابتدا معاملات بعد از ۲۰۱۰ را انتخاب کنید، سپس از resample استفاده کنید و در نهایت تعداد خانه‌هایی که فاصله آنها کمتر از ۱۵ کیلومتر است را بشمارید.</li>

</ul>
</font>
</p>

<p style="text-align:center;"><img src="fig2.png"></p>



In [ ]:

fig2 , ax2 = None

<h2 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
<b>سلول جواب‌ساز</b>
</font>
</h2>


In [ ]:

import zipfile
import joblib

joblib.dump(districts_df,"districts_df")
joblib.dump(fig1,"fig1")
joblib.dump(fig2,"fig2")

def compress(file_names):
    print("File Paths:")
    print(file_names)
    # Select the compression mode ZIP_DEFLATED for compression
    # or zipfile.ZIP_STORED to just store the file
    compression = zipfile.ZIP_DEFLATED
    # create the zip file first parameter path/name, second mode
    with zipfile.ZipFile("result.zip", mode="w") as zf:
        for file_name in file_names:
            # Add file to the zip file
            # first parameter file to zip, second filename in zip
            zf.write('./' + file_name, file_name, compress_type=compression)


file_names = ["districts_df","fig1","fig2", '5_time_series.ipynb']
compress(file_names)